In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import os
import os.path
import random
import numpy as np
import random
import requests
from tensorflow.python.framework import ops
import pandas as pd

In [0]:
df = pd.read_csv('drive/My Drive/signate/takeda/dataset/train.csv')
dftest = pd.read_csv('drive/My Drive/signate/takeda/dataset/test.csv')

print(df.shape)
print(dftest.shape)

(13731, 3807)
(13732, 3806)


In [0]:
df_Score = df['Score']
df = df.drop('Score', axis=1)
dftest_id = dftest['ID']

print(df.shape)
print(dftest.shape)

(13731, 3806)
(13732, 3806)


In [0]:
# reset the graph for new run
ops.reset_default_graph()

# Create graph session 
sess = tf.Session()

# set batch size for training
batch_size = 100

# make results reproducible
seed = 3
np.random.seed(seed)
tf.set_random_seed(seed)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#x_vals_train, x_vals_test, y_vals_train, y_vals_test = train_test_split(df.as_matrix(), df_Score.as_matrix(), test_size=0.2, random_state=0)

# 標準化(重回帰時のみ使用)
#sc = StandardScaler()
#sc.fit(X_train)
#x_vals_train = sc.transform(X_train)
#x_vals_test = sc.transform(X_test)

#print(x_vals_train.shape)
#print(y_vals_train.shape)
#print(x_vals_test.shape)
#print(y_vals_test.shape)
x_vals_train = df.as_matrix()
y_vals_train = df_Score.as_matrix()

print(x_vals_train.shape)
print(y_vals_train.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(13731, 3806)
(13731,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [0]:
# Record training column max and min for scaling of non-training data
train_max = np.max(x_vals_train, axis=0)
train_min = np.min(x_vals_train, axis=0)

# Normalize by column (min-max norm to be between 0 and 1)
def normalize_cols(mat, max_vals, min_vals):
    return (mat - min_vals) / (max_vals - min_vals)

x_vals_train = np.nan_to_num(normalize_cols(x_vals_train, train_max, train_min))
#x_vals_test = np.nan_to_num(normalize_cols(x_vals_test, train_max, train_min))
  
print(x_vals_train.shape)
print(y_vals_train.shape)
#print(x_vals_test.shape)
#print(y_vals_test.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


(13731, 3806)
(13731,)


In [0]:
# Define Variable Functions (weights and bias)
def init_weight(shape, st_dev):
    weight = tf.Variable(tf.random_normal(shape, stddev=st_dev))
    return(weight)
    
def init_bias(shape, st_dev):
    bias = tf.Variable(tf.random_normal(shape, stddev=st_dev))
    return(bias)
    
# Create Placeholders
x_data = tf.placeholder(shape=[None, 3806], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

In [0]:
# Create a fully connected layer:
def fully_connected(input_layer, weights, biases):
    layer = tf.add(tf.matmul(input_layer, weights), biases)
    return(tf.nn.relu(layer))

def final_connected(input_layer, weights, biases):
    layer = tf.add(tf.matmul(input_layer, weights), biases)
    return layer
  
#--------Create the first layer (50 hidden nodes)--------
weight_1 = init_weight(shape=[3806, 10], st_dev=10.0)
bias_1 = init_bias(shape=[10], st_dev=10.0)
layer_1 = fully_connected(x_data, weight_1, bias_1)

#--------Create second layer (25 hidden nodes)--------
weight_2 = init_weight(shape=[10, 5], st_dev=10.0)
bias_2 = init_bias(shape=[5], st_dev=10.0)
layer_2 = fully_connected(layer_1, weight_2, bias_2)

#--------Create third layer (5 hidden nodes)--------
weight_3 = init_weight(shape=[5, 3], st_dev=10.0)
bias_3 = init_bias(shape=[3], st_dev=10.0)
layer_3 = fully_connected(layer_2, weight_3, bias_3)

#--------Create output layer (1 output value)--------
weight_4 = init_weight(shape=[3, 1], st_dev=10.0)
bias_4 = init_bias(shape=[1], st_dev=10.0)
final_output = final_connected(layer_3, weight_4, bias_4)

# Declare loss function (L1)
loss = tf.reduce_mean(tf.abs(y_target - final_output))

# Declare optimizer
my_opt = tf.train.AdamOptimizer(0.025)
train_step = my_opt.minimize(loss)

In [0]:
# Initialize Variables
init = tf.global_variables_initializer()
sess.run(init)

# Training loop
loss_vec = []
test_loss = []
for i in range(80000):
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    rand_x = x_vals_train[rand_index]
    rand_y = np.transpose([y_vals_train[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})

    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
    
    #test_temp_loss = sess.run(loss, feed_dict={x_data: x_vals_test, y_target: np.transpose([y_vals_test])})
    #test_loss.append(test_temp_loss)
    
    if (i+1) % 20000 == 0:
        print('Generation: ' + str(i+1) + '. Loss = ' + str(temp_loss))
        #print('Generation: ' + str(i+1) + '. Loss = ' + str(temp_loss) + '. TEST_LOSS ' + str(test_temp_loss))

Generation: 20000. Loss = 0.74229693
Generation: 40000. Loss = 0.749016
Generation: 60000. Loss = 0.77937585
Generation: 80000. Loss = 0.78952545


In [0]:
x_vals_test = dftest.as_matrix()

# 標準化(重回帰時のみ使用)
#sc = StandardScaler()
#sc.fit(XX)
#XX_train = sc.transform(XX)

# Record training column max and min for scaling of non-training data
#test_max = np.max(x_vals_test, axis=0)
#test_min = np.min(x_vals_test, axis=0)

# Normalize by column (min-max norm to be between 0 and 1)
x_vals_test = np.nan_to_num(normalize_cols(x_vals_test, train_max, train_min))

print(x_vals_train.shape)
print(y_vals_train.shape)
print(x_vals_test.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


(13731, 3806)
(13731,)
(13732, 3806)


In [0]:
YY = sess.run(final_output, feed_dict={x_data: x_vals_train})
print(YY.shape)
dftest_Score = pd.DataFrame(YY)

dftest_Score = dftest_Score.astype(float)
print(dftest_Score.shape)
print(dftest_Score.isnull().any())
dftest_Score = pd.concat(objs=[dftest_id, dftest_Score], axis=1)
dftest_Score

(13731, 1)
(13731, 1)
0    False
dtype: bool


,ID,0
0,1,1.972265
1,2,1.972265
2,3,1.972265
3,4,1.972265
4,6,1.972265
5,7,1.972265
6,10,1.972265
7,11,1.972265
8,13,1.972265
9,16,1.972265


In [0]:
dftest_Score.to_csv('submit_NN.csv', index=False, header=False)

In [0]:
# 欠損値の処理
#df_missing_value = pd.DataFrame()
#dftest_missing_value = pd.DataFrame()

#for flag, name in zip(df.isnull().any(), df.columns):
#  if flag == True:
#    df_missing_value = pd.concat(objs=[df_missing_value, df[name]], axis=1)
#    dftest_missing_value = pd.concat(objs=[dftest_missing_value, dftest[name]], axis=1)
#    df = df.drop(name, axis=1)
#    dftest = dftest.drop(name, axis=1)
    
#for flag, name in zip(dftest.isnull().any(), dftest.columns):
#  if flag == True:
#    df_missing_value = pd.concat(objs=[df_missing_value, df[name]], axis=1)
#    dftest_missing_value = pd.concat(objs=[dftest_missing_value, dftest[name]], axis=1)
#    dftest = dftest.drop(name, axis=1)
#    df = df.drop(name, axis=1)

#same_colums = []
#same_columns = df_missing_value.columns & dftest_missing_value.columns
#print(len(same_colums))
#print(df_missing_value.shape)
#print(dftest_missing_value.shape)

In [0]:
#%matplotlib inline
# Plot loss (MSE) over time
#plt.plot(loss_vec, 'k-', label='Train Loss')
#plt.plot(test_loss, 'r--', label='Test Loss')
#plt.title('Loss (MSE) per Generation')
#plt.legend(loc='upper right')
#plt.xlabel('Generation')
#plt.ylabel('Loss')
#plt.show()

In [0]:
# Model Accuracy
#actuals = np.array([x[0] for x in birth_data])
#test_actuals = actuals[test_indices]
#train_actuals = actuals[train_indices]
#test_preds = [x[0] for x in sess.run(final_output, feed_dict={x_data: x_vals_test})]
#train_preds = [x[0] for x in sess.run(final_output, feed_dict={x_data: x_vals_train})]
#test_preds = np.array([1.0 if x < 2500.0 else 0.0 for x in test_preds])
#train_preds = np.array([1.0 if x < 2500.0 else 0.0 for x in train_preds])
# Print out accuracies
#test_acc = np.mean([x == y for x, y in zip(test_preds, test_actuals)])
#train_acc = np.mean([x == y for x, y in zip(train_preds, train_actuals)])
#print('On predicting the category of low birthweight from regression output (<2500g):')
#print('Test Accuracy: {}'.format(test_acc))
#print('Train Accuracy: {}'.format(train_acc))

In [0]:
# Initialize Variables
#init = tf.global_variables_initializer()
#sess.run(init)

# Training loop
#loss_vec = []
#test_loss = []
#for i in range(100000):
#    rand_index = np.random.choice(len(X), size=batch_size)
#    rand_x = x_vals_test[rand_index]
#    rand_y = np.transpose([y_vals_test[rand_index]])
#    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})